## Different Demos on How To Build Agents

### Basic AgentWorkflow

In [4]:
!pip install -U llama-index-llms-google-genai llama-index google-genai dotenv

#### Set Global variables

In [ ]:
from dotenv import load_dotenv

import os

load_dotenv()

MODEL="gemini-2.5-flash" # @param ["gemini-3-pro-preview", "gemini-3-flash-preview", "gemini-2.5-flash", "gemini-2.5-flash-lite", "gemini-2.5-pro"]
PROJECT_ID=os.getenv("GCP_PROJECT_ID")
LOCATION=os.getenv("GCP_LOCATION")

1. FunctionAgent

In [ ]:
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.core.agent.workflow import FunctionAgent

def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b


def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b

llm = GoogleGenAI(
    model=MODEL,
    vertexai_config={"project": PROJECT_ID, "location": LOCATION}
)

math_agent = FunctionAgent(
    tools=[multiply, add],
    name="MathAgent",
    description="An agent that can perform basic mathematical operations using the multiply and add tools.",
    llm=llm,
    system_prompt="You are an agent that can perform basic mathematical operations using tools. In addition to the result, also list what tools you used with the input parameters."
)

user_msg = "What is 20+2*4?"

response = await math_agent.run(user_msg=user_msg)
print(response)

20+2*4 = 28
Used tools:
default_api.multiply(a=2, b=4)
default_api.add(a=20, b=8)



2. AgentWorkflow

In [3]:
from llama_index.core.agent.workflow import AgentWorkflow

workflow = AgentWorkflow.from_tools_or_functions(
    [multiply, add],
    llm=llm,
    system_prompt="You are an agent that can perform basic mathematical operations using tools. In addition to the result, also list what tools you used with the input parameters."
)

response = await workflow.run(user_msg=user_msg)
print(response)

20 + 2 * 4 = 28
Used tools:
default_api.multiply(a=2, b=4)
default_api.add(a=20, b=8)



### Using existing tools

In [25]:
!pip install llama-index-tools-yahoo-finance

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 29.1 MB/s  0:00:00 eta 0:00:01
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15634 sha256=21d9850a2124460d50fa4a1f494adb1615b4f03324711b352bd989eb1a08478e
  Stored in directory: /home/baliga/.cache/pip/wheels/cc/bd/6f/664d62c99327abeef7d86489e6631cbf45b56fbf7ef1d6ef00
Successfully built multitasking
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [llama-index-tools-yahoo-finance]


In [4]:
from llama_index.tools.yahoo_finance import YahooFinanceToolSpec

my_tools = YahooFinanceToolSpec().to_tool_list()
my_tools.extend([multiply, add])

finance_agent = FunctionAgent(
    tools=my_tools,
    name="FinanceAgent",
    description="Useful for performing financial operations.",
    llm=llm,
    system_prompt="You are an agent that can perform basic mathematical operations and financial data retrieval using tools. In addition to the result, also list what tools you used with the input parameters."
)

user_msg = "How much will I pay in total if I buy 10 shares of Apple and 5 shares of Nvidia at the current stock price?"

response = await finance_agent.run(user_msg=user_msg)
print(response)

You will pay a total of $3708.25.

Tools used:
default_api.stock_basic_info(ticker='AAPL')
default_api.stock_basic_info(ticker='NVDA')
default_api.multiply(a=10, b=278.12)
default_api.multiply(a=5, b=185.41)
default_api.add(a=2781.2, b=927.05)



### Maintaining state

By default, the *AgentWorkflow* is stateless between runs. This means that the agent will not have any memory of previous runs.

To maintain state, we need to keep track of the previous state. In LlamaIndex, Workflows have a __Context__ class that can be used to maintain state within and between runs. Since the AgentWorkflow is just a pre-built Workflow, we can also use it now.

In [9]:
from llama_index.core.workflow import Context
from llama_index.core.workflow import JsonPickleSerializer, JsonSerializer

workflow = AgentWorkflow.from_tools_or_functions(
    my_tools,
    llm=llm,
    system_prompt="You are an agent that can perform basic mathematical operations and financial data retrieval using tools."
)

# configure a context to work with our workflow
myctx = Context(workflow)

response = await workflow.run(user_msg="Hi, my name is Laurie!",ctx=myctx)
print(response)

response2 = await workflow.run(user_msg="What's my name?",ctx=myctx)
print(response2)

# convert our Context to a dictionary object
ctx_dict = myctx.to_dict(serializer=JsonSerializer())

# create a new Context from the dictionary
restored_ctx = Context.from_dict(
    workflow, ctx_dict, serializer=JsonSerializer()
)

response3 = await workflow.run(user_msg="What's my name?",ctx=restored_ctx)
print(response3)


Hello Laurie! How can I help you today?

I'm sorry, I don't have memory of past conversations. I only know that you just told me your name is Laurie. 

I do not have access to past conversations, so I do not know your name.
